# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from sklearn.impute import SimpleImputer
from patsy import dmatrices

In [ ]:
df = pd.read_csv('previsao_de_renda.csv')
df.fillna(value=0, inplace=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          15000 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [ ]:
# 1 - Preparação dos dados
df['log_renda'] = np.log(df['renda'])  # Transformação logarítmica para a variável dependente
X = df.drop(columns=['renda','log_renda', 'data_ref', 'id_cliente'])  # Removendo variáveis irrelevantes
y = df['log_renda']

# Divisão da base em treinamento (75%) e teste (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Pré-processamento: transformação de variáveis categóricas em dummies
categorical_features = X.select_dtypes(include=['object', 'bool']).columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

# Definição do ColumnTransformer para transformação de dados
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

X_train = X_train.dropna()
y_train = y_train.loc[X_train.index]
X_test = X_test.dropna()
y_test = y_test.loc[X_test.index]


In [ ]:
# 2 - Modelos de Regressão Ridge e Lasso
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
best_ridge_score = -np.inf
best_ridge_model = None

for alpha in alphas:
    ridge = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', Ridge(alpha=alpha))])
    ridge.fit(X_train, y_train)
    ridge_pred = ridge.predict(X_test)
    ridge_r2 = r2_score(y_test, ridge_pred)
    print(f'Alpha: {alpha}, R2 Score: {ridge_r2}')

    if ridge_r2 > best_ridge_score:
        best_ridge_score = ridge_r2
        best_ridge_model = ridge

print(f'Melhor modelo Ridge com alpha={best_ridge_model["regressor"].alpha}, R2: {best_ridge_score}')

Alpha: 0, R2 Score: 0.3505045296211817
Alpha: 0.001, R2 Score: 0.35050453567305306
Alpha: 0.005, R2 Score: 0.3505045598460189
Alpha: 0.01, R2 Score: 0.35050458998489886
Alpha: 0.05, R2 Score: 0.35050482806918337
Alpha: 0.1, R2 Score: 0.3505051184209472
Melhor modelo Ridge com alpha=0.1, R2: 0.3505051184209472


In [ ]:
#3 - Modelo Lasso
best_lasso_score = -np.inf
best_lasso_model = None

for alpha in alphas:
    lasso = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', Lasso(alpha=alpha))])
    lasso.fit(X_train, y_train)
    lasso_pred = lasso.predict(X_test)
    lasso_r2 = r2_score(y_test, lasso_pred)
    print(f'Alpha: {alpha}, R2 Score: {lasso_r2}')

    if lasso_r2 > best_lasso_score:
        best_lasso_score = lasso_r2
        best_lasso_model = lasso

print(f'Melhor modelo LASSO com alpha={best_lasso_model["regressor"].alpha}, R2: {best_lasso_score}')


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.873e+03, tolerance: 8.797e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

Alpha: 0, R2 Score: 0.35049867530947687
Alpha: 0.001, R2 Score: 0.3511602092428856
Alpha: 0.005, R2 Score: 0.35033198099020324
Alpha: 0.01, R2 Score: 0.3463347904214438
Alpha: 0.05, R2 Score: 0.30710281248530247
Alpha: 0.1, R2 Score: 0.2452291097950634
Melhor modelo LASSO com alpha=0.001, R2: 0.3511602092428856


In [ ]:
# 4.1 - Stepwise

# Preparação do dataframe
df_stepwise = df.drop(columns=['data_ref', 'renda'])  # Removendo colunas irrelevantes

df_stepwise.columns = df_stepwise.columns.str.replace(' ', '_')

# Evitar erros na stepwise

# Remover a coluna 'Unnamed: 0' se ela existir
df_stepwise = df_stepwise.drop(columns=['Unnamed: 0'], errors='ignore')

# Remoção de valores nulos
df_stepwise.dropna(inplace=True)

# Transformação de variáveis categóricas
colunas_categoricas = df_stepwise.select_dtypes(include=['object']).columns
df_stepwise = pd.get_dummies(df_stepwise, columns=colunas_categoricas, drop_first=True)

# Transformação de variáveis booleanas para inteiros
colunas_bool = df_stepwise.select_dtypes(include=['bool']).columns
df_stepwise[colunas_bool] = df_stepwise[colunas_bool].astype(int)

df_stepwise.columns = df_stepwise.columns.str.replace(' ', '_')



# Definição das variáveis para Stepwise Selection
X_stepwise = df_stepwise.drop(columns=['id_cliente', 'log_renda'])
y_stepwise = df_stepwise['log_renda']

# Conferencia das colunas
print(df_stepwise.columns,'\n')



# Função de Stepwise Selection
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out=0.05,
                       verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(X_stepwise, y_stepwise)

print('resulting features:')
print(result)

Index(['Unnamed:_0', 'id_cliente', 'posse_de_veiculo', 'posse_de_imovel',
       'qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia',
       'log_renda', 'sexo_M', 'tipo_renda_Bolsista', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'tipo_renda_Servidor_público',
       'educacao_Pós_graduação', 'educacao_Secundário',
       'educacao_Superior_completo', 'educacao_Superior_incompleto',
       'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_União',
       'estado_civil_Viúvo', 'tipo_residencia_Casa',
       'tipo_residencia_Com_os_pais', 'tipo_residencia_Comunitário',
       'tipo_residencia_Estúdio', 'tipo_residencia_Governamental'],
      dtype='object') 

Add  sexo_M                         with p-value 0.0
Add  tempo_emprego                  with p-value 0.0
Add  tipo_renda_Pensionista         with p-value 1.97679e-72
Add  tipo_renda_Empresário          with p-value 5.36735e-25
Add  educacao_Superior_completo     with p-value 6.38984e-22
Add  

In [ ]:
# 4.2 - Atualizando a formula
formula = 'log_renda ~ ' + ' + '.join(result)

# Patsy
y_stepwise, X_stepwise = dmatrices(formula, data=df_stepwise, return_type='dataframe')
model = sm.OLS(y_stepwise, X_stepwise).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     797.2
Date:                Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                        13:39:52   Log-Likelihood:                -16226.
No. Observations:               15000   AIC:                         3.247e+04
Df Residuals:                   14989   BIC:                         3.256e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

**5 -**

O modelo **LASSO** com *alpha = 0.001* parece oferecer o melhor desempenho em termos de R-quadrado, sugerindo que pode ser o mais adequado para seus dados com base nas métricas fornecidas.

Contudo.

O modelo **OLS** é mais simples e não envolve regularização, o que pode levar ao sobreajuste se o modelo for muito complexo.

In [ ]:
# 6 - O R² na Regressão ridge aparenta estar aumentando conforme a gente aumenta o alpha
# vou testar com valores maiores

alphas = [0.5, 0.6, 0.7, 0.8]
best_ridge_score = -np.inf
best_ridge_model = None

for alpha in alphas:
    ridge = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', Ridge(alpha=alpha))])
    ridge.fit(X_train, y_train)
    ridge_pred = ridge.predict(X_test)
    ridge_r2 = r2_score(y_test, ridge_pred)
    print(f'Alpha: {alpha}, R2 Score: {ridge_r2}')

    if ridge_r2 > best_ridge_score:
        best_ridge_score = ridge_r2
        best_ridge_model = ridge

print(f'Melhor modelo Ridge com alpha={best_ridge_model["regressor"].alpha}, R2: {best_ridge_score}')

Alpha: 0.5, R2 Score: 0.3505072080714482
Alpha: 0.6, R2 Score: 0.35050768096981333
Alpha: 0.7, R2 Score: 0.35050813967348027
Alpha: 0.8, R2 Score: 0.35050858650725447
Melhor modelo Ridge com alpha=0.8, R2: 0.35050858650725447


In [ ]:
# 7 Ajuste da árvore de regressão
tree_model = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', DecisionTreeRegressor(random_state=81))])
tree_model.fit(X_train, y_train)
tree_pred = tree_model.predict(X_test)
tree_r2 = r2_score(y_test, tree_pred)

print(f'R² da Árvore de Regressão: {tree_r2}')

R² da Árvore de Regressão: -0.15018958910704838
